In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from scipy import signal
import os.path

2021-11-18 12:39:37.567447: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-18 12:39:37.567471: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
output = ['normal','impaired']
trial = ['01','02','03']
subjectInitial = 1  
subjectFinal = 25     #specify from which subject to which subject
initString = './Smartphone1/subject'    # Keep the folder 'Smartphone1' from where you are running this python file

### Function to read Gyro data given filepath

In [3]:
def readData(fileString):
    global time , gyr_x ,gyr_y, gyr_z 
    data = pd.read_csv(fileString)
    if 'Time (s)' in data:
        time = data['Time (s)'].values
    if 'Gyroscope x (rad/s)' in data:  
        gyr_x = data['Gyroscope x (rad/s)'].values
    elif 'X (rad/s)'  in data:          # some excelfiles have headers as this
        gyr_x = data['X (rad/s)'].values
        
    if 'Gyroscope y (rad/s)' in data:    
        gyr_y = data['Gyroscope y (rad/s)'].values
    elif 'Y (rad/s)'  in data:
        gyr_y = data['Y (rad/s)'].values  
        
    if 'Gyroscope z (rad/s)' in data:        
        gyr_z = data['Gyroscope z (rad/s)'].values
    elif 'Z (rad/s)'  in data:
        gyr_z = data['Z (rad/s)'].values
    #print(len(time) ,'',len(gyr_x),'',len(gyr_y),'',len(gyr_z)  )    
    #return (time , gyr_x , gyr_y , gyr_z)

### Clip data using determined start and end timestep

In [4]:
def clipData(clipStart , clipEnd):
    global time , gyr_x ,gyr_y, gyr_z
    time  = time[clipStart : clipEnd]
    gyr_x = gyr_x[clipStart : clipEnd]
    gyr_y = gyr_y[clipStart : clipEnd]
    gyr_z = gyr_z[clipStart : clipEnd]

### Filtering the data

In [5]:
def filterData(sampFreq):
    global gyr_x ,gyr_y, gyr_z
    order  = 5
    cutOffFreq = 5
    b, a = signal.butter(order, cutOffFreq*2/sampFreq, 'low')
    gyr_x = signal.filtfilt(b, a, gyr_x)
    gyr_y = signal.filtfilt(b, a, gyr_y)
    gyr_z = signal.filtfilt(b, a, gyr_z)

### Plot Data

In [7]:
def plotData(x,y,label):
    plt.plot(x, y, label=label)
    plt.grid()
    plt.show()

### Resample Data

In [8]:
def resampleData(numSamples,clipDuration):
    global time , gyr_x ,gyr_y, gyr_z
    time = np.linspace(0, clipDuration, numSamples)
    gyr_x = signal.resample(gyr_x, numSamples)
    gyr_y = signal.resample(gyr_y, numSamples)
    gyr_z = signal.resample(gyr_z, numSamples)

### Export Data

In [9]:
def exportData(filePath):
    global time , gyr_x ,gyr_y, gyr_z
    filteredData = {'Time (s)': time,
        'Gyroscope x (rad/s)': gyr_x,
        'Gyroscope y (rad/s)': gyr_y,
        'Gyroscope z (rad/s)': gyr_z
        }
    df = pd.DataFrame(filteredData, columns= ['Time (s)', 'Gyroscope x (rad/s)','Gyroscope y (rad/s)','Gyroscope z (rad/s)'])
    df.to_csv (filePath+'/filteredGyro.csv',index = False, header=True)

## Looping over and preprocessing all dataset 

In [10]:
for i in range(subjectInitial,subjectFinal+1):
    subnum =''
    if i<10:
        subnum= '0'
    subnum+=str(i)+'_' 
    for j in range(2):
        for k in range(3):
            filePath = initString + subnum + output[j] +trial[k]  #complete file path 
            print(filePath)
            if os.path.isfile(filePath+'/Gyroscope.csv'): #check if file path exists
                time , gyr_x ,gyr_y, gyr_z= [],[],[],[] #initializing all lists as empty 
                readData(filePath+'/Gyroscope.csv') # getRawData
                if len(time)==0 or len(gyr_x)==0 or len(gyr_y)==0 or len(gyr_z)==0: # data was not found for given header
                    print('not read')
                    continue 
                print('read')    
                totalStep = len(time)
                duration = time[totalStep-1] - time[0]
                sampPeriod = (duration)/totalStep #sampling period
                sampFreq = 1/sampPeriod #sampling freq
                sampFreq = int(round(sampFreq)) #rounding off to an integer
                midTime= int(time[0] +(duration/2)+5)
                clipDuration =20  # can be decided based on requirement 
                clipStart = int(midTime-clipDuration/2)*sampFreq
                clipEnd = int(midTime+clipDuration/2)*sampFreq
                clipData(clipStart , clipEnd) # clipData
                print('clipped')
                filterData(sampFreq)
                print('filtered')
                numSamples =4500 # to be decided
                resampleData(numSamples,clipDuration)
                print('resampled')
                exportData(filePath)
                print('exported')
            else:
                print('file path not found')
                continue
                
    

./Smartphone1/subject01_normal01
file path not found
./Smartphone1/subject01_normal02
file path not found
./Smartphone1/subject01_normal03
file path not found
./Smartphone1/subject01_impaired01
file path not found
./Smartphone1/subject01_impaired02
file path not found
./Smartphone1/subject01_impaired03
file path not found
./Smartphone1/subject02_normal01
file path not found
./Smartphone1/subject02_normal02
file path not found
./Smartphone1/subject02_normal03
file path not found
./Smartphone1/subject02_impaired01
file path not found
./Smartphone1/subject02_impaired02
file path not found
./Smartphone1/subject02_impaired03
file path not found
./Smartphone1/subject03_normal01
file path not found
./Smartphone1/subject03_normal02
file path not found
./Smartphone1/subject03_normal03
file path not found
./Smartphone1/subject03_impaired01
file path not found
./Smartphone1/subject03_impaired02
file path not found
./Smartphone1/subject03_impaired03
file path not found
./Smartphone1/subject04_norm